In [1]:
# packages for data processing
import pandas as pd
import re

In [2]:
# read in yelp business data 
BUSINESS_DATA_PATH = '/Users/nikolaj/Desktop/datathon/yelp_dataset/yelp_academic_dataset_business.json'

business_df = pd.read_json(BUSINESS_DATA_PATH, lines=True)
print(business_df.shape)
business_df.head()

(150346, 14)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [3]:
# extract Cafes/Coffee/Tea/Bubble Tea shops in California, Nevada
desired_categories = ['cafes', 'cafe' 'coffee', 'tea', 'bubble tea']
pattern = r'\b(?:' + '|'.join(map(re.escape, desired_categories)) + r')\b'
cafe_df = business_df[((business_df['state'] == 'CA') | (business_df['state'] == 'NV')) & 
                      (business_df['categories'].str.contains(pattern, case=False, regex=True))]
print(cafe_df.shape)
cafe_df.head()

(590, 14)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
85,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,CA,93101,34.414445,-119.690672,4.0,389,1,"{'RestaurantsTakeOut': 'True', 'NoiseLevel': '...","Food, Restaurants, Salad, Coffee & Tea, Breakf...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-14:0', '..."
470,VeFfrEZ4iWaecrQg6Eq4cg,Cal Taco,"7320 Hollister Ave, Ste 1",Goleta,CA,93117,34.430542,-119.882367,4.0,189,1,"{'RestaurantsGoodForGroups': 'True', 'Business...","Burgers, Cafes, Restaurants, Mexican, American...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-20:30', ..."
555,bdfZdB2MTXlT6-RBjSIpQg,Pho Bistro,903 Embarcadero Del Norte,Isla Vista,CA,93117,34.412934,-119.855531,3.0,184,1,"{'RestaurantsDelivery': 'True', 'BikeParking':...","Food, Restaurants, Chinese, Bubble Tea, Vietna...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
691,-kY_HDP7IMvGl-kBIZVU4A,Dune Coffee Roasters - Anacapa,528 Anacapa St,Santa Barbara,CA,93101,34.418994,-119.695068,4.0,320,1,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...","Coffee & Tea, Coffee Roasteries, Food","{'Monday': '6:0-17:0', 'Tuesday': '6:0-17:0', ..."
766,3vTG6LQyr3obzfvrK1l1hA,7-Eleven,15 Eagle Canyon Dr,Sparks,NV,89441,39.638503,-119.712869,2.0,12,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Food, Gas Stations, Service Stations, Convenie...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."


In [ ]:
REVIEW_DATA_PATH = '/Users/nikolaj/Desktop/datathon/yelp_dataset/yelp_academic_dataset_review.json'

# Get the list of business_ids you're interested in
cafe_ids = set(cafe_df['business_id'].unique())

# We'll store matching reviews in this list
filtered_reviews = []

# Read and filter in chunks

ENTRY_CAP = 1000
chunk_counter = 1
with open(REVIEW_DATA_PATH, 'r') as f:
    for line in f:
        review = pd.read_json(line, lines=True)
        if review['business_id'][0] in cafe_ids:
            filtered_reviews.append(review)

        if len(filtered_reviews) == ENTRY_CAP:
            filtered_review_df = pd.concat(filtered_reviews, ignore_index=True)

            output_path = f'/Users/nikolaj/Desktop/datathon/review_chunks/filtered_reviews_chunk_{chunk_counter}.csv'
            filtered_review_df.to_csv(output_path, index=False)
            print(f"Wrote chunk {chunk_counter} with {len(filtered_review_df)} reviews to {output_path}")

            # Clear memory
            filtered_reviews = []
            chunk_counter += 1

if filtered_reviews:
    filtered_review_df = pd.concat(filtered_reviews, ignore_index=True)
    output_path = f'/Users/nikolaj/Desktop/datathon/review_chunks/filtered_reviews_chunk_{chunk_counter}.csv'
    filtered_review_df.to_csv(output_path, index=False)
    print(f"Wrote final chunk {chunk_counter} with {len(filtered_review_df)} reviews to {output_path}")



/var/folders/4j/sk4xv0ws3vqf9k27myg6288m0000gn/T/ipykernel_2093/252888140.py:15: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  review = pd.read_json(line, lines=True)
/var/folders/4j/sk4xv0ws3vqf9k27myg6288m0000gn/T/ipykernel_2093/252888140.py:15: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  review = pd.read_json(line, lines=True)
/var/folders/4j/sk4xv0ws3vqf9k27myg6288m0000gn/T/ipykernel_2093/252888140.py:15: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  review = pd.read_json(line, lines=True)
/var/folders/4j/sk4xv0ws3vqf9k27myg6288m0000gn/T/ipykernel_2093/252888140.py:15: FutureWarning: Passing literal json to 'rea

OSError: Cannot save file into a non-existent directory: '/Users/nikolaj/Desktop/datathon/review_chunks'

In [6]:
print(filtered_reviews)
print(len(filtered_reviews))

[                review_id                 user_id             business_id  \
0  L0jv8c2FbpWSlfNC6bbUEA  bFPdtzu11Oi0f92EAcjqmg  IDtLPgUrqorrpqSLdfMhZQ   

   stars  useful  funny  cool  \
0      5       0      0     0   

                                                text                date  
0  What a great addition to the Funk Zone!  Grab ... 2016-10-13 22:50:47  ,                 review_id                 user_id             business_id  \
0  zM7u3iffMjniyvQAaN9Tnw  QacYrFyCrFLmC726YEEBsA  29YqJwOGEuAWqlHZxMc1OA   

   stars  useful  funny  cool  \
0      1       0      1     0   

                                                text                date  
0  So disappointing on so many levels. Have been ... 2012-05-29 03:51:08  ,                 review_id                 user_id             business_id  \
0  Gczff1Qxg4WVmlBLPrZwzQ  cpp89UW0imv45y3a_O444w  34Eqv8jXgxg_EEwcsNgeeg   

   stars  useful  funny  cool  \
0      5       0      0     0   

                               

In [ ]:
variable_name = 'cafe_df'
variable_value = cafe_df

# Define the Python file path (same folder as the notebook)
file_path = './cafe_df.py'

# Open the file and write the variable into it
with open(file_path, 'w') as f:
    f.write(f"{variable_name} = {variable_value}\n")